# 1) Données

In [9]:
import pandas as pd
import numpy as np
import csv
import os

# calcul de la position dans le roulement d'un poste en fonction de la semaine
def rFunction(s,p):
    return ((p-s)%24)

# calcul du numéro de semaine correspondant au jour j
def sFunction(j):
    return j//7

# calcul du jour non travaillé pour une semaine donnée d'un opérateur à 80%
# en fonction du jour non travaillé la première semaine
def delta_i(s,d_i):
    return (d_i+s)%5

# operateurs + competences
dataOperateurs = pd.read_csv('./data/operateurs.csv',sep = ";" )

cardO = len(dataOperateurs.axes[0])
colsDataOperateurs = len(dataOperateurs.axes[1])
nomsOperateurs = dataOperateurs.iloc[0:cardO,0:3]
competencesOperateurs = dataOperateurs.iloc[0:cardO,3:(colsDataOperateurs+1)]

# postes (intitulés + rho)
dataPostes = pd.read_csv('./data/postes.csv',sep = ";" )
cardP = len(dataPostes.axes[0])
nomsPostes = dataPostes.iloc[0:cardO,1]
rho = [0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1]

# le nombre de semaines <=> nombre de postes <=> nombre d'opérateurs
#cardS = cardO
cardS = 4
cardJ = 7*cardS
cardR = cardO

# kappa_ip (opérateur possède la compétence pour le poste)
#kappa = pd.read_csv('./data/kappa_ip.csv',sep = ";" )
kappa = [
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,0,0,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0]]

# sigma_pp' (postes sur le même créneau horaire)
#sigma = pd.read_csv('./data/sigma_pp.csv',sep = ";" )
sigma= [
[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],
[0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1]]

d = [3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]

# o_j (jour ouvrés)
o = np.zeros(cardJ)
for j in range(0,len(o)):
    if (j%7 <= 4):
        o[j] = 1

# H_s (jours de chaque semaine)
H = []
for i in range(0,cardS):
    H.append(np.zeros(7))
for i in range(0,cardS):
    for j in range(0,7):
        H[i][j]=(i*7 + j)
        
profils = np.zeros(cardO)
cardProfils = 0
for i in range(0,cardO):
    found = False
    for j in range(0,i):
        if not found:
            diff = False
            if d[i] == d[j]:
                for p in range(0,cardP):
                    if kappa[i][p] != kappa[j][p]:
                        diff = True
            else:
                diff = True
        
            if not diff:
                profil = profils[j]
                found = True
    if found:
        profils[i] = profil
    else:
        profils[i] = cardProfils
        cardProfils = cardProfils + 1
        
        
#print(profils)
#print(cardProfils)
N = np.zeros(cardProfils)
for i in range(cardO):
    N[int(profils[i])] += 1
#print(N)

# delta_ij (est-ce qu'une personne à 80% travaille le jour j)
#delta = pd.read_csv('./data/delta_ij.csv',sep = ";" )
# d[i] représente le jour non travaillé de l'opérateur i la première semaine, d[i]=-1 si i est à temps plein

delta = np.full((cardProfils, cardJ), 1)
for profil in range(cardProfils):
    found = False
    i = 0
    index = -1
    while not found and i < cardO:
        if profils[i] == profil:
            found = True
            index = i
        i += 1
    if d[index] != -1:
        d_i = d[index]
        for s in range(0, cardS):
            d_is = delta_i(s,d_i) #rang du jour non travaillé la semaine s
            j_s = 7*s + d_is      #numéro du jour non travaillé
            delta[profil][j_s] = 0
            
kappaProfils = []
for profil in range(cardProfils):
    found = False
    i = 0
    index = -1
    while not found and i < cardO:
        if profils[i] == profil:
            found = True
            index = i
        i += 1
    kappaProfils.append(kappa[index])
    
creneaux = [0,2,1,2,0,1,3,1,0,2,1,2,0,1,3,1,0,1,2,1,0,3,1,2,0]
cardCreneaux = len(np.unique(creneaux))
alpha = []
for p in range(cardP):
    ligne = []
    for c in range(cardCreneaux):
        if creneaux[p] == c:
            ligne.append(1)
        else:
            ligne.append(0)
    alpha.append(ligne)

#print(nomsOperateurs)
#print(competencesOperateurs)
#print(nomsPostes)
#print(rho)
#print(kappa)
#print(sigma)
print(delta)
#print(o)
#print(H[1])
#print(kappaProfils)
#print(delta[0])

[[1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]


# 2) Modèle mathématique


## 2.1) Variables

In [2]:
from pulp import *

# x_ijp = 1 si l'opérateur prend en charge le poste p le jour j
x = {} 
for prof in range(0,cardProfils):
    for j in range(0,cardJ): 
        for p in range(0,cardP): 
            x[prof,j,p]=LpVariable(f"x({prof,j,p})", cat=LpBinary) #LpBinary

# y_ir = 1 si l'opérateur i est position r dans le roulement
y = {} 
for prof in range(0,cardProfils):
    for r in range(0,cardR):    
        y[prof,r]=LpVariable(f"y({prof,r})", cat=LpBinary)
        
# z_isp = 1 si l'opérateur est affecté au poste p la semaine s
z = {} 
for prof in range(0,cardProfils):
    for s in range(0,cardS): 
        for p in range(0,cardP): 
            z[prof,s,p]=LpVariable(f"z({prof,s,p})", cat=LpBinary)

# u_is = 1 si l'opérateur i est affecté à un poste différent de celui prévu la semaine s
u = {} 
for p in range(0,cardP):
    for s in range(0,cardS):    
        u[p,s]=LpVariable(f"u({p,s})", cat=LpBinary)

print("nb de variables x_ijp = ", len(x))
print("nb de variables y_ir  = ", len(y))
print("nb de variables z_isp = ", len(z))
print("nb de variables u_ir  = ", len(u))
print()
print("nb total de variables = ", len(x)+len(y)+len(z)+len(u))

nb de variables x_ijp =  8400
nb de variables y_ir  =  120
nb de variables z_isp =  1200
nb de variables u_ir  =  240

nb total de variables =  9960


## 2.2) Création du modèle avec PuLP

In [3]:
prob = LpProblem("ModeleProfils", LpMinimize)

prob += sum(u[p,s] for p in range(0,cardP) for s in range(0,cardS)), "obj"

## 2.3) Contraintes

In [4]:
#contrainte 1 Un profil opérateur ne peut prendre un poste que s'il est compétent et disponible sur un jour ouvré

for i in range(0,cardProfils):
    for j in range(0,cardJ):
        for p in range(0,cardP):
            prob += (x[i,j,p] <= delta[i][j]*kappaProfils[i][p]*o[j], f"C1_{i,j,p}")

#contrainte 2 Un profil opérateur ne peut être affecté à un poste une semaine que s'il en possède la compétence

for i in range(0,cardProfils):
    for s in range(0,cardS):
        for p in range(0,cardP):
            prob += (z[i,s,p] <= kappaProfils[i][p], f"C2_{i,s,p}")

#contrainte 3 Chaque jour, un profil opérateur est affecté à son poste de la semaine (s'il n'est pas rouleur)
for i in range(0,cardProfils):
    for s in range(0,cardS):
        for j in H[s]:
            for p in range(0,cardP):
                prob += x[i,j,p] <= z[i,s,p] + sum((1 - rho[p2])*z[i,s,p2] for p2 in range(0,cardP)), f"C3_{i,j,p}"

#contrainte 4 Chaque jour, un profil opérateur est affecté à autant de poste que son nombre de membres

for i in range(0,cardProfils):
    for j in range(0,cardJ):
        if o[j] == 1 and delta[i][j] == 1:
            prob += sum(x[i,j,p] for p in range(0,cardP)) == N[i], f"C4_{i,j}"

#contrainte 5 Chaque jour, chaque poste non rouleur est assuré par exactement un profil opérateur
for p in range(0,cardP):
    for j in range(0,cardJ):
        if o[j] == 1 and rho[p] == 1:
            prob += sum(x[i,j,p] for i in range(cardProfils)) == 1, f"C5_{j,p}"
            
#contrainte 6 Chaque jour, les postes rouleurs sont occupés par au plus un profil opérateur
for p in range(0,cardP):
    for j in range(0,cardJ):
        if o[j] == 1 and rho[p] == 0:
            prob += sum(x[i,j,p] for i in range(0,cardProfils)) <= 1, f"C6_{j,p}"

#contrainte 7 Chaque semaine, chaque profil opérateur est affecté à autant de poste que son nombre de membres

for i in range(0,cardProfils):
    for s in range(0,cardS):
        prob += sum(z[i,s,p] for p in range(0,cardP)) == N[i], f"C7_{i,s}"

#contrainte 8 Chaque semaine, chaque poste est assuré par un profil opérateur

for p in range(0,cardP):
    for s in range(0,cardS):
        prob += sum(z[i,s,p] for i in range(0,cardProfils)) == 1, f"C8_{s,p}"

#contrainte 9 Chaque profil opérateur est affecté à autant de positions dans le roulement que son nombre de membres

for i in range(0,cardProfils):
    prob += sum(y[i,r] for r in range(0,cardR)) == N[i], f"C9_{i}"

#contrainte 10 Chaque position dans le roulement est associée à un profil opérateur

for r in range(0,cardR):
    prob += sum(y[i,r] for i in range(0,cardProfils)) == 1, f"C10_{r}"

#contrainte 11 Un profil opérateur peut permuter de poste si leurs horaires sont compatibles

for i in range(0,cardProfils):
    for s in range(0,cardS):
        for p in range(0,cardP):
            prob += z[i,s,p] <= sum(sigma[p][p2]*y[i,rFunction(s,p2)] for p2 in range(0,cardP)), f"C11_{i,s,p}"

#contrainte 12 Lorsqu'un poste est occupé par un autre profil opérateur que celui prévu initialement,
#son score d'insatisfaction augmente
for s in range(0,cardS):
    for p in range(0,cardP):
        for i in range(cardProfils):
            prob += u[p,s] >= (kappaProfils[i][p]*rho[p]*y[i,rFunction(s,p)] - z[i,s,p]), f"C12_{i,s,p}"
            
#contrainte David anti permutation interdite
for s in range(cardS):
    for i in range(cardProfils):
        for c in range(cardCreneaux):
            prob += sum(alpha[p][c]*z[i,s,p] for p in range(cardP)) == sum(alpha[p][c]*y[i,rFunction(s,p)] for p in range(cardP)), f"C13_{s,i,c}"


## 2.4) Résolution du problème

In [5]:
import time
import cplex
from cplex.callbacks import MIPInfoCallback
log = []

class MyCallback(MIPInfoCallback):

    def __call__(self):
        current = time.time()
        log.append([self.get_incumbent_objective_value(), current - start])

pathFile = "Modeles/ModeleExactProfils.lp"
prob.writeLP(pathFile)

c = cplex.Cplex()
c.parameters.timelimit.set(60.0)
c.register_callback(MyCallback)
c.read(pathFile)

start = time.time()
c.solve()
#solution = c.solution
#print(solution.get_objective_value())
end = time.time()

duree = end - start
print()
print("Temps de résolution : " + str(duree))
print("Valeur objectif = ", c.solution.get_objective_value())

print()
print("log")
for i in range(len(log)):
    print(log[i])



Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               60
Legacy callback                                  i
Tried aggregator 2 times.
MIP Presolve eliminated 14110 rows and 3360 columns.
MIP Presolve modified 1440 coefficients.
Aggregator did 10 substitutions.
Reduced MIP has 8239 rows, 6590 columns, and 58290 nonzeros.
Reduced MIP has 6590 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.09 sec. (75.75 ticks)
Found incumbent of value 180.000000 after 0.17 sec. (120.56 ticks)
Probing changed sense of 300 constraints.
Probing time = 0.07 sec. (40.49 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 410 rows and 300 columns.
Reduced MIP has 7829 rows, 6290 columns, and 56790 nonzeros.
Reduced MIP has 6290 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.09 sec. (77.01 ticks)
Probing time = 0.02 sec. (7.02 ticks)
Clique table m

# 3) Affichage de la solution

In [6]:
print("objectif =", c.solution.get_objective_value())
for s in range(cardS):
    for p in range(cardP):
        val = c.solution.get_values("u(("+str(p)+",_"+str(s)+"))")
        if val != 0:
            print("u[",p,",",s,"] =", val)

print()
for i in range(cardProfils):
    for r in range(cardR):
        val = c.solution.get_values("y(("+str(i)+",_"+str(r)+"))")
        if val != 0:
            print("y[",i,",",r,"] =", val)
        
print()
for s in range(cardS):
    print()
    for i in range(cardProfils):
        for p in range(cardP):
            val = c.solution.get_values("z(("+str(i)+",_"+str(s)+",_"+str(p)+"))")
            if val != 0:
                print("z[",i,",",s,",",p,"] =", val)

#print(c.variables.get_names())
#solver.solution.get_objective_value()
#print(objval)

objectif = 0.0

y[ 0 , 20 ] = 1.0
y[ 1 , 8 ] = 1.0
y[ 1 , 9 ] = 1.0
y[ 1 , 10 ] = 1.0
y[ 1 , 11 ] = 1.0
y[ 1 , 12 ] = 1.0
y[ 1 , 13 ] = 1.0
y[ 1 , 14 ] = 1.0
y[ 1 , 15 ] = 1.0
y[ 1 , 16 ] = 1.0
y[ 1 , 17 ] = 1.0
y[ 1 , 18 ] = 1.0
y[ 1 , 21 ] = 1.0
y[ 1 , 22 ] = 1.0
y[ 2 , 4 ] = 1.0
y[ 2 , 5 ] = 1.0
y[ 2 , 6 ] = 1.0
y[ 2 , 7 ] = 1.0
y[ 2 , 23 ] = 1.0
y[ 3 , 0 ] = 1.0
y[ 3 , 1 ] = 1.0
y[ 3 , 2 ] = 1.0
y[ 3 , 3 ] = 1.0
y[ 4 , 19 ] = 1.0


z[ 0 , 0 , 20 ] = 1.0
z[ 1 , 0 , 8 ] = 1.0
z[ 1 , 0 , 9 ] = 1.0
z[ 1 , 0 , 10 ] = 1.0
z[ 1 , 0 , 11 ] = 1.0
z[ 1 , 0 , 12 ] = 1.0
z[ 1 , 0 , 13 ] = 1.0
z[ 1 , 0 , 14 ] = 1.0
z[ 1 , 0 , 15 ] = 1.0
z[ 1 , 0 , 16 ] = 1.0
z[ 1 , 0 , 17 ] = 1.0
z[ 1 , 0 , 18 ] = 1.0
z[ 1 , 0 , 21 ] = 1.0
z[ 1 , 0 , 22 ] = 1.0
z[ 2 , 0 , 4 ] = 1.0
z[ 2 , 0 , 5 ] = 1.0
z[ 2 , 0 , 6 ] = 1.0
z[ 2 , 0 , 7 ] = 1.0
z[ 2 , 0 , 23 ] = 1.0
z[ 3 , 0 , 0 ] = 1.0
z[ 3 , 0 , 1 ] = 1.0
z[ 3 , 0 , 2 ] = 1.0
z[ 3 , 0 , 3 ] = 1.0
z[ 4 , 0 , 19 ] = 1.0

z[ 0 , 1 , 21 ] = 1.0
z[ 1 , 1 , 9 ] = 1.

# 4) Extraction de la solution au format texte

In [7]:
with open("solutionsTexte/solutionModeleExactProfils", "w") as file_object:
        file_object.write(str(cardO) + "\n")           #cardO
        file_object.write(str(cardP) + "\n")           #cardP
        file_object.write(str(cardR) + "\n")           #cardR
        file_object.write(str(cardS) + "\n")           #cardS
        ligne = ""
        for p in range(0,cardP):
            ligne = ligne + str(rho[p])
        file_object.write(ligne + "\n")                #rho
        
        ligne = ""
        for j in range(0,cardJ):
            ligne = ligne + str(int(o[j]))
        file_object.write(ligne + "\n")                #o_j
        
        for i in range(0,cardO):
            ligne = ""
            for p in range(0,cardP):
                ligne = ligne + str(kappa[i][p])
            file_object.write(ligne + "\n")            #kappa_ip
            
        for p in range(0,cardP):
            ligne = ""
            for p2 in range(0,cardP):
                ligne = ligne + str(sigma[p][p2])
            file_object.write(ligne + "\n")            #sigma_pp'
            
        for i in range(0,cardO):
            ligne = ""
            for j in range(0,cardJ):
                ligne = ligne + str(delta[i][j])
            file_object.write(ligne + "\n")            #delta_ij
        
        for i in range(0,cardO):
            for j in range(0,cardJ):
                ligne = ""
                for p in range(0,cardP):
                    ligne = ligne + str(int(x[i,j,p].varValue))
                file_object.write(ligne + "\n")        #x_ijp
        
        for i in range(0,cardO):
            ligne = ""
            for r in range(0,cardR):
                ligne = ligne + str(int(y[i,r].varValue))
            file_object.write(ligne + "\n")            #y_ir

        for i in range(0,cardO):
            for s in range(0,cardS):
                ligne = ""
                for p in range(0,cardP):
                    ligne = ligne + str(int(z[i,s,p].varValue))
                file_object.write(ligne + "\n")        #z_isp
        
        file_object.write(str(int(value(prob.objective))) + "\n")

IndexError: index 5 is out of bounds for axis 0 with size 5

# 5) Extraction de la solution au format CSV

In [10]:
import csv
with open('CSV/planningExactProfils.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    jours = [""]
    for s in range(0,cardS):
        jours.extend(["lundi","mardi","mercredi","jeudi","vendredi","samedi","dimanche"])
    writer.writerow(jours)
    
    dates = [""]
    for j in range(0,cardJ):
        dates.append("jour "+str(j))
    writer.writerow(dates)
    
    for i in range(0,cardO):
        planningPerso = [nomsOperateurs.iloc[i,1] + " " + nomsOperateurs.iloc[i,2]]
        for j in range(0,cardJ):
            found = False
            for p in range(0,cardP):
                if x[i,j,p].varValue == 1:
                    found = True
                    planningPerso.append(nomsPostes.iloc[p])
            if not found:
                if o[j] == 0:
                    planningPerso.append("")
                else:
                    planningPerso.append("RA")
        writer.writerow(planningPerso)

KeyError: (5, 0, 0)

In [ ]:
print("Fin de programme")